In [1]:
# https://github.com/jiecaoyu/pytorch-nin-cifar10
# https://github.com/yangqiongyongyu/Network-In-Network-Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
from torch.utils import data
from torchvision import transforms
import importlib.util
spec = importlib.util.spec_from_file_location("module.name", "/home/arnab/Desktop/dnn-offloading/Models/bdddataloader.py")
bddloader = importlib.util.module_from_spec(spec)
spec.loader.exec_module(bddloader)

in_channel = np.zeros((2),dtype=int) # worker = 2
in_channel[0] = 3
in_channel[1] = 3
def node(img_part,current_layer,BATCH_SIZE,channel,model,fog_node):
    global in_channel
    out_channel = np.zeros((2),dtype=int) # worker = 2
    weight = None
    bias = None
    layer_kernel = None
    layer_stride = None
    layer_padding = None
    cfgs = [192, 160, 96, 'M', 192, 192, 192, 'A', 192, 192, 12, 'A']
    kernel_filters = [5,1,1,3,5,1,1,3,3,1,1,8]
    stride = [1,1,1,2,1,1,1,2,1,1,1,1]
    padding = [2,0,0,1,2,0,0,1,1,0,0,0]
    model_list = create_model_list(model)
    model_dict = create_model_dict(cfgs,model_list,kernel_filters,stride,padding)
    
        
    if len(model_dict[current_layer]) > 4:
        cfg = model_dict[current_layer][0]
        out_channel[fog_node] = model_dict[current_layer][0]
        layer_kernel = model_dict[current_layer][1]
        layer_stride = model_dict[current_layer][2]
        layer_padding = model_dict[current_layer][3]
        weight = model[model_dict[current_layer][4]]
        bias = model[model_dict[current_layer][5]]
    else:
        cfg = model_dict[current_layer][0]
        layer_kernel = model_dict[current_layer][1]
        layer_stride = model_dict[current_layer][2]
        layer_padding = model_dict[current_layer][3]
        
    dropout = nn.Dropout(0.5)

    if cfg == 'M':
        print("\tBefore MaxPool: {}".format(img_part.size()))
        m = nn.MaxPool2d(kernel_size=layer_kernel, stride=layer_stride, padding=layer_padding)
        print("\t" + str(m))
        return dropout(m(img_part))
            
    elif cfg == 'A':
        print("\tBefore AvgPool: {}".format(img_part.size()))
        m = nn.AvgPool2d(kernel_size=layer_kernel, stride=layer_stride, padding=layer_padding)
        print("\t" + str(m))
        if current_layer != (len(cfgs)-1):
            return dropout(m(img_part))
        else:
            return m(img_part)
    else:
        print("\tBefore Conv: {}".format(img_part.size()))
        m0 = nn.Conv2d(in_channel[fog_node], out_channel[fog_node], kernel_size=layer_kernel, stride=layer_stride, padding=layer_padding)
        m1 = nn.ReLU(inplace=True)
        print("\t" + str(m0))
        m0.weight.data = weight
        m0.bias.data = bias
        in_channel[fog_node] = out_channel[fog_node]
        return m1(m0(img_part))
        
    #print(m.state_dict().keys())

def create_model_list(model):
    model_list = []
    for key in model:
        model_list.append(key)
        
    return model_list

def create_model_dict(cfg,model_list,kernel_filters,stride,padding):
    model_dict = {}
    model_inx = 0
    for i,v in enumerate(cfg):
        if v == 'M' or v == 'A':
            model_dict.update({i:(v,kernel_filters[i],stride[i],padding[i])})
        else:
            model_dict.update({i:(v,kernel_filters[i],stride[i],padding[i],model_list[model_inx],model_list[model_inx+1])})
            model_inx += 2
            
    return model_dict

def adaptive_partitioning(img,partition_size):
    index = partition_size
    temp = img.detach().numpy()
    temp = torch.from_numpy(temp[:,:,index[0]:index[1],:])
    return temp

def partition(NN,img,k,CA,f):
    # intializing variables
    A = None
    partition_size = []
    batch = img.shape[0]
    channel = img.shape[1]
    W = [0] * k  # partition
    Windex = []  # partitioning indexes
    init = [0] * (k + 1)
    CA[0] = 0
    CA = [float(val) for val in CA]

    r = img.shape[2]
    c = img.shape[3]
    
    if r > c:
        m = r
    else:
        m = c
    
    # sum of capabilities of all nodes
    C2 = np.sum(CA[:(k + 1)])
    
    for i in range(1,k+1):
        C1 = np.sum(CA[:i+1])
        Pi = C1 / C2
        if NN == "conv":
            init[i] = math.floor(Pi * (m - (f - 1)))
            partition_size.append((init[i-1],init[i]+(f-1)))
        elif NN == "ff":
            init[i] = math.floor(Pi * m)
            partition_size.append((init[i - 1],init[i]))

    return partition_size


def main():
    # Configuration of network
    cfgs = [192, 160, 96, 'M', 192, 192, 192,'A', 192, 192, 12, 'A']
    
    # Load model
    model = torch.load("/home/arnab/Desktop/Data/nin.pt", map_location=torch.device('cpu'))
    
    # Number of workers
    worker = 2
    
    # Capabilities of nodes
    #CA = np.random.randint(1, 10, size=worker+1) 
    CA = [0.0, 4.0, 6.0] # Capabilities of nodes
    print("Fog CA: {}".format(CA))
    
    # Initialize variables
    current_layer = None
    classify_list = []
    after_part = None
    input_ = None
    channel = None
    final_out = None
    BATCH_SIZE = 1
    IMAGE_DIM = 32
    kernel_filters = [5,1,1,3,5,1,1,3,3,1,1,8]
    
    # load images
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(IMAGE_DIM),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
    
    loader = data.DataLoader(
        bddloader.BDDDataset('/home/arnab/Desktop/Data', train=True, transform=transform),
        batch_size=BATCH_SIZE,
        shuffle=True)
    
    inx = 0
    for img,level in loader:
        print(f"Input image size: {img.size()}")
        print("=> Image : {}".format(inx+1))
        
        # Convolutional NN
        for i in range(len(cfgs)):
            current_layer = i
            if i == 0:
                input_ = img
                channel = input_.shape[1]
            else:
                input_ = final_out
                channel = input_.shape[1]

            print("=> Layers: {}".format(i))
            after_part = partition("conv",input_,worker,CA,kernel_filters[current_layer])

            # processing and marging
            final_out = None
            for j in range(len(after_part)):
                img_part = adaptive_partitioning(input_,after_part[j])
                out = node(img_part,current_layer,BATCH_SIZE,channel,model,j)
                print("\tAfter: {}\n".format(out.size()))
                if j == 0:
                    final_out = out
                else:
                    final_out = torch.cat((final_out,out),2)
                    
            print("\tAfter Marge: " + str(final_out.size()))
        
        avgpool = nn.AdaptiveAvgPool2d((1, 1))
        final_out = avgpool(final_out)
        #final_out = final_out.view(final_out.size(0), 12)
        final_out = torch.flatten(final_out,1)
        m = nn.ReLU()
        out = m(final_out).data > 0
        out = out.int()
        print(f"Final output: {out.size()}")
        
        inx += 1
        break # image break
    
    out.size()

if __name__ == '__main__':
    main()

Fog CA: [0.0, 4.0, 6.0]
Input image size: torch.Size([1, 3, 32, 32])
=> Image : 1
=> Layers: 0
	Before Conv: torch.Size([1, 3, 15, 32])
	Conv2d(3, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
	After: torch.Size([1, 192, 15, 32])

	Before Conv: torch.Size([1, 3, 21, 32])
	Conv2d(3, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
	After: torch.Size([1, 192, 21, 32])

	After Marge: torch.Size([1, 192, 36, 32])
=> Layers: 1
	Before Conv: torch.Size([1, 192, 14, 32])
	Conv2d(192, 160, kernel_size=(1, 1), stride=(1, 1))
	After: torch.Size([1, 160, 14, 32])

	Before Conv: torch.Size([1, 192, 22, 32])
	Conv2d(192, 160, kernel_size=(1, 1), stride=(1, 1))
	After: torch.Size([1, 160, 22, 32])

	After Marge: torch.Size([1, 160, 36, 32])
=> Layers: 2
	Before Conv: torch.Size([1, 160, 14, 32])
	Conv2d(160, 96, kernel_size=(1, 1), stride=(1, 1))
	After: torch.Size([1, 96, 14, 32])

	Before Conv: torch.Size([1, 160, 22, 32])
	Conv2d(160, 96, kernel_size=(1, 1), stride=(1, 1))
	After